# GDP vs VACCINATION

In this project we seek to analyze the GDP for selected countries depending on the child vaccination rates. 

Imports and set magics:

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataproject


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Read and clean data

We have imported data in cvs-files from the OECD Databank. 

**The 1st dataset:**
The first dataset is an overview of GDP per capita for 34 countries in the world from 2018 to 2022. The included variables are year, GDP per capita and country. 

**The 2nd dataset:**
The second dataset is an overview of child vaccination rates for XX countries in the world from 2018 to 2022. The included variables are year, GDP per capita and country. 

In [40]:
# import data
vac = pd.read_csv('vaccination.csv') 
# inspect data
print(vac) 

    LOCATION    INDICATOR  SUBJECT   MEASURE FREQUENCY  TIME  Value Flag Codes
0        AUS  CHILDVACCIN      DTP  PC_CHILD         A  2018   94.6        NaN
1        AUS  CHILDVACCIN      DTP  PC_CHILD         A  2019   94.9        NaN
2        AUS  CHILDVACCIN      DTP  PC_CHILD         A  2020   95.4        NaN
3        AUS  CHILDVACCIN      DTP  PC_CHILD         A  2021   95.0        NaN
4        AUS  CHILDVACCIN  MEASLES  PC_CHILD         A  2018   95.0          E
..       ...          ...      ...       ...       ...   ...    ...        ...
319      PER  CHILDVACCIN      DTP  PC_CHILD         A  2019   88.0        NaN
320      PER  CHILDVACCIN      DTP  PC_CHILD         A  2020   72.2        NaN
321      PER  CHILDVACCIN  MEASLES  PC_CHILD         A  2018   85.0        NaN
322      PER  CHILDVACCIN  MEASLES  PC_CHILD         A  2019   85.0        NaN
323      PER  CHILDVACCIN  MEASLES  PC_CHILD         A  2020   77.1        NaN

[324 rows x 8 columns]


In [41]:
# Dropping the unecessary colums
drop_these = ['FREQUENCY', 'Flag Codes', 'INDICATOR', 'MEASURE']
vac.drop(drop_these, axis=1, inplace=True)
# Renaming columns
vac.rename(columns = {'LOCATION':'country', 'TIME':'year', 'SUBJECT':'subject', 'Value':'cv_rate'}, inplace=True)
vac.head()

,country,subject,year,cv_rate
0,AUS,DTP,2018,94.6
1,AUS,DTP,2019,94.9
2,AUS,DTP,2020,95.4
3,AUS,DTP,2021,95.0
4,AUS,MEASLES,2018,95.0


In [42]:
# Reshaping from long to wide
vac = vac.pivot_table(index='country', columns='year', values='cv_rate').reset_index() 
vac.head()

year,country,2018,2019,2020,2021
0,ARG,90.0,84.50,75.8,78.35
1,AUS,94.8,94.95,95.2,93.85
2,AUT,89.5,89.50,89.5,NaN
3,BEL,97.0,97.00,96.5,NaN
4,BRA,89.5,80.50,78.3,NaN


In [43]:
# Removing variables where data is missing
vac = vac.dropna()
print(vac)

year country   2018   2019   2020   2021
0        ARG  90.00  84.50  75.80  78.35
1        AUS  94.80  94.95  95.20  93.85
8        CHN  98.85  98.95  99.30  99.40
9        COL  93.50  94.50  89.20  86.45
10       CRI  94.00  95.00  95.80  90.50
11       CZE  96.70  94.55  95.60  95.15
13       DNK  95.00  95.50  94.80  94.95
14       ESP  95.25  96.75  96.15  92.60
15       EST  89.40  89.60  90.90  89.45
18       GBR  93.00  92.00  92.05  91.75
20       HUN  99.90  99.90  99.85  99.85
21       IDN  92.30  95.80  86.60  83.65
22       IND  94.85  92.95  86.90  87.10
23       IRL  93.40  92.25  93.00  91.95
24       ISL  92.00  92.60  93.15  91.80
29       LTU  92.25  92.40  90.75  89.25
31       LVA  97.00  99.50  99.45  95.45
32       MEX  92.50  77.50  79.40  83.45
33       NLD  93.20  93.65  93.90  93.75
34       NOR  96.00  97.00  96.90  96.70
35       NZL  92.25  91.70  91.75  90.45
39       RUS  97.25  97.30  96.90  97.00
40       SVK  96.30  96.45  96.50  96.20
43       TUR  97

## Explore each data set

In order to be able to **explore the raw data**, you may provide **static** and **interactive plots** to show important developments 

**Interactive plot** :

In [44]:
def plot_func():
    # Function that operates on data set
    pass

widgets.interact(plot_func, 
    # Let the widget interact with data through plot_func()    
); 


interactive(children=(Output(),), _dom_classes=('widget-interact',))

Explain what you see when moving elements of the interactive plot around. 

# Merge data sets

Now you create combinations of your loaded data sets. Remember the illustration of a (inner) **merge**:

In [45]:
plt.figure(figsize=(15,7))
v = venn2(subsets = (4, 4, 10), set_labels = ('Data X', 'Data Y'))
v.get_label_by_id('100').set_text('dropped')
v.get_label_by_id('010').set_text('dropped' )
v.get_label_by_id('110').set_text('included')
plt.show()

NameError: name 'venn2' is not defined

<Figure size 1500x700 with 0 Axes>

Here we are dropping elements from both data set X and data set Y. A left join would keep all observations in data X intact and subset only from Y. 

Make sure that your resulting data sets have the correct number of rows and columns. That is, be clear about which observations are thrown away. 

**Note:** Don't make Venn diagrams in your own data project. It is just for exposition. 

# Analysis

To get a quick overview of the data, we show some **summary statistics** on a meaningful aggregation. 

MAKE FURTHER ANALYSIS. EXPLAIN THE CODE BRIEFLY AND SUMMARIZE THE RESULTS.

# Conclusion

ADD CONCISE CONLUSION.